# Sincronizar CAT y lista local de agrobd

In [1]:
import json
import pandas as pd
import requests

# local import
from paths import *

In [33]:
def getInfoTaxon(record_id):
    '''
    Hace una consulta a zacatuche para obtener la información que se necesita de un id.
    Devuelve una pandas.Series con la información solicitada en la query.
    '''
    query = """query taxon{
                dwcTaxon(taxonID:"""+"\""+record_id+"\""+"""){
                            id
                            taxonomicStatus
                            scientificName
                            acceptedNameUsage{
                            id
                            scientificName
                            }
                        }
                        }"""
    
    # ¿conviene renombrar todas las columnas? Sí
    New_col_names = {'acceptedNameUsage.id': 'id_valido',
                     'acceptedNameUsage.scientificName' : 'taxon_valido'
                    }

    path_zacatuche = 'http://zacatuche2.conabio.gob.mx:4000/graphql'
    r = requests.post(path_zacatuche, json={'query': query})

    # TO DO: check that response.status = 200, else print error to log

    json_data = json.loads(r.text)

    # case when id is not in CAT
    if json_data['data']['dwcTaxon'] is None:
        return None

    # case when there is no id_valido associated to id
    if json_data['data']['dwcTaxon']['acceptedNameUsage'] is None:
        df_data = pd.json_normalize(json_data['data']['dwcTaxon'])
        df_data[['id_valido', 'taxon_valido']] = None, None
        return df_data.loc[0]
    
    df_data = pd.json_normalize(json_data['data']['dwcTaxon'])
    df_data = (df_data.rename(columns = New_col_names)
                          .fillna(''))
    
    return df_data.loc[0]

In [34]:
test = getInfoTaxon('44557ANGIOx')

In [35]:
test

In [48]:
def updateLocal(agrobd_id, New_values):
    '''
    Realiza las modificaciones en la base de catalogo-agrobiodiversidad de los campos que se le pasen como parámetro.
    '''
    New_values['usuario'] = 'Bot validación'
    
    # ¿Cómo deben aparecer los valores vacíos? 
    new_values = ''  
    for field in New_values:
        new_values += f'{field}: \"{New_values[field]}\" \n'

    query = f'''mutation{{
                updateAgrobiodiversidad(
                    id:"{agrobd_id}"
                    {new_values}){{
                    id
                }}
                }}'''
    requests.post(paths.siagro, json={'query': query})

In [8]:
def is_synonym(record):  
    return (record['id_valido']) and (record['id'] != record['id_valido'])

In [ ]:
def request_agrobd_review(record):
    # if record doesn't have agrobd label, request review
    if not record['categoria_agrobiodiversidad'] == 'Agrobiodiversidad':
        note = record['comentarios_revision'] + '\n REVISAR ETIQUETA AGROBIODIVERSIDAD'
        updateLocal(record['id'], {'comentarios_revision': note})

In [ ]:
def delete_agrobd_label(agrobd):
    # TO DO: also delete subcategoria_agrobiodiversidad
    # TO DO: cómo poner campos vacíos    
    updateLocal(agrobd['id'], {'categoria_agrobiodiversidad': ''})

In [ ]:
def id_is_in_agrobd_list(record_id, agrobd_list):
    # TO DO: test this func
    return record_id in agrobd_list['id']

In [ ]:
def add_id_to_agrobd_list(record):
    query = f'''mutation{{
                addAgrobiodiversidad(
                    id:"{record['id']}"
                    taxon:"{record['taxon']}"
                    estatus:"{record['estatus']}"
                    id_valido:"{record['id_valido']}"
                    taxon_valido:"{record['taxon_valido']}"){{
                    id
                }}
                }}'''
    requests.post(paths.siagro, json={'query': query})

In [ ]:
def add_agrobd_label(record):
    # TO DO: add subcategoria_agrobiodiversidad y referencias
    updateLocal(record['id'], {'categoria_agrobiodiversidad': 'Agrobiodiversidad'})

In [ ]:
def change(agrobd, catalog):
    # sync estatus, id_valido, taxon_valido
    updateLocal(agrobd['id'], {'estatus': catalog['taxonomicStatus'],
                    'id_valido': catalog['id_valido'],
                    'taxon_valido': catalog['taxon_valido']
                    })

    # sinonimo to valido OR NA to valido
    if agrobd['id'] == catalog['id_valido']:
        request_agrobd_review(agrobd)
    
    # valido to NA
    elif (agrobd['id'] == agrobd['id_valido']) and (not catalog['id_valido']):
        delete_agrobd_label(agrobd)
    
    # sinonimo to sinonimo
    elif is_synonym(agrobd) and is_synonym(catalog):
        if not id_is_in_agrobd_list(catalog['id_valido']):
            add_id_to_agrobd_list(catalog['id_valido'])
            request_agrobd_review(catalog['id_valido'])
    
    # valido to sinonimo
    elif (agrobd['id'] == agrobd['id_valido']) and is_synonym(catalog):
        delete_agrobd_label(agrobd['id'])
        if not id_is_in_agrobd_list(catalog['id_valido']):
            add_id_to_agrobd_list(catalog['id_valido'])
        
        add_agrobd_label(catalog['id'])
    

In [ ]:
def sync_agrobd_to_catalog(agrobd_list):

    sin_taxon_valido_asociado = []

    for i, agrobd in agrobd_list.iterrows():
        if 'pendiente' in agrobd['id']:
            continue

        catalog = getInfoTaxon(agrobd['id'])

        if catalog is None:
            sin_taxon_valido_asociado.append(agrobd['id'])
            continue

        if agrobd['id_valido'] != catalog['id_valido']:
            # identify transition
            
            # changes
            pass

        if agrobd['taxon'] != catalog['scientificName']:
            updateLocal(agrobd['id'], {'taxon': catalog['taxon_valido']})
    
    sendeMail(sin_taxon_valido_asociado)
